In [ ]:
import numpy as np
import time
import glob
import os

class Library:
    
    def __init__(self, nb_books, signup_day, ship_day):
        self.nb_books = nb_books
        self.signup_day = signup_day
        self.ship_day = ship_day

class Solution:

    def __init__(self, lib_id, scan_nr, book_ids):
        self.lib_id = lib_id
        self.scan_nr = scan_nr
        self.book_ids = book_ids

    
def optimizer():

    # Calculate individual score of each library that measures how valuable the library is for scanning
    score_sums = np.array([np.sum(scores[lib.lbooks]) for lib in libraries]) # Get the score list for book ids of each library and sum up
    average_score = score_sums / nb_books_list

    library_score = np.zeros([2, total_libraries])
    
    """ Optimization Strategy """
    # The higher the average book score the more the library can contribute to the overall score
    # The higher the ship_day value the more books per day can be scanned -> nominator
    # The lower the signup_day value the faster the scanning can start -> denominator
    library_score[0,:] = average_score * ship_day_list / signup_day_list
    library_score[1,:] = np.arange(0, total_libraries)
    
    sorted_index = np.flip(library_score[0,:].argsort())
    sorted_libraries = library_score[1, sorted_index]
    #sorted_index and sorted_libraries have the same function. Shouldn't we juste write:
    # sorted_libraries = np.flip(library_score[0,:].argsort())
    # return sorted_libraries 
    #(Laura)
    # I don't get what you mean.
    
    return sorted_libraries.astype(np.int16)

def allocate_libraries(sorted_libraries):
    overallScore = 0
    day_offset = 0
    lookup_index = 0
    lib_nr = len(sorted_libraries)
    lib_solutions = []

    for day in range (total_days):
        # print(f"day: {day}")
        
        j = sorted_libraries[lookup_index]
        if lookup_index < lib_nr: # if there are unsigned libraries left
            if (libraries[j].signup_day - 1 + day_offset <= day): # -1 because the library needs to be signed up on the day before scanning starts

                #print(f"\t lib: {j} signed up")

                # init score sorting for library
                lib_score_list = np.zeros([2, libraries[j].nb_books], dtype=np.uint32) #The type should be int64, otherwise, you will have random value(Laura) Yes, it's a bit small.
                lib_score_list[0,:] = scores[libraries[j].lbooks]
                lib_score_list[1,:] = libraries[j].lbooks
                lib_score_index = np.flip(lib_score_list[0,:].argsort()) # Return sorted indices of book scores in library
                lib_score_list = lib_score_list[:, lib_score_index]

                # add offset for the next library signup
                day_offset += libraries[j].signup_day

                # calculate how many books can be scanned in the remaining days
                days_left = total_days - day - 1
                scan_book_nr = days_left * libraries[j].ship_day
                # print(f"ship day: {libraries[j].ship_day}")
                # print(f"scan_book_nr. {scan_book_nr}")
                
                # filter books that have been scanned already
                lib_scanned_books = scanned_books[lib_score_list[1,:]] # bool values that indicates if a book in the library has been scanned already
                filtered_scan_list = lib_score_list[:,lib_scanned_books] # filtered list
                scanned_books[filtered_scan_list[1,:]] = 0 # update scanned book list
                #print(scanned_books)

                ##ALTERNATIVE IF WE INITIALIZE scanned_books with false(Laura):
                #lib_scanned_books = scanned_books[lib_score_list[1,:]] # bool values that indicate if a book in the library has been scanned already
                #filtered_scan_list = lib_score_list[:,[not elem for elem in lib_scanned_books]] # filtered list
                #scanned_books[filtered_scan_list[1,:]] = 1 # update scanned book list

                # @Laura Why would you want to solve it with a for loop if you can sove it in numpy which is usually faster?
                # I don't get your point here.

                
                # Reduce scan list so it fits into number of remaining days
                if filtered_scan_list.shape[1] > scan_book_nr: # in case there are more books than days left for scanning
                    filtered_scan_list = filtered_scan_list[:, 0:scan_book_nr]
                
                # Calculate score sums
                lib_score_sum = filtered_scan_list[0,:].sum()
                # print(f"\t score: {lib_score_sum}")
                overallScore += lib_score_sum

                # Collect submission infos
                # lib_id: j
                # number of scanned books: filtered_scan_list length
                # book_ids: filtered_scan_list row 2 with all the ids
                sol = Solution(j, filtered_scan_list.shape[1], filtered_scan_list[1,:])
                lib_solutions.append(sol)
                
                # increment j for next library
                lookup_index += 1
            else:
                pass
                # print(f"\t lib: {j} signing up ...")
        else:
            pass
            # print("no libraries left")

    solutions = {"signup_nr": lookup_index, "lib_solutions": lib_solutions}
    
    print(f"Score: {overallScore}")
    return solutions

def create_submission(solutions):

    with open(f"Submissions/sub_{file_name}",'w') as file:

        # overwrite potential existing lines
        file.write("")

        # write number of signed up libraries
        print(str(solutions["signup_nr"]), file=file)
        
        # iterated over signed up library objects
        for sol in solutions["lib_solutions"]:
            string1 = str(sol.lib_id) + " " + str(sol.scan_nr)
            string2 = ' '.join(map(str, sol.book_ids))
            print(string1, file=file)
            print(string2, file=file)


for path in glob.glob("Datasets/*.txt"):
    # Parsing
    file_name = os.path.basename(path)
    with open(path,'r') as file: 
        
        # get values 
        nb_books, total_libraries, total_days = [int(x) for x in next(file).split()]
        scores = np.array([int (x) for x in next(file).split()])

        # Preallocate arrays that will be used for optimization later
        nb_books_list = np.zeros([total_libraries]) 
        signup_day_list = np.zeros([total_libraries])
        ship_day_list = np.zeros([total_libraries])
        libraries = [] # list of all Library objects

        #get the information for each library:
        #nb_books: number total of books
        #signup_day: number of day needed to sign up
        #ship_day: number of books per day 
        #lbooks: list of the books in the library
        for i in range(total_libraries):
            lib_nb_books, signup_day, ship_day = [int(x) for x in next(file).split()]
            
            # add library specific information to arrays for optimization
            nb_books_list[i] = lib_nb_books
            signup_day_list[i] = signup_day
            ship_day_list[i] = ship_day
            
            # add library specific information to Library object
            lib_info = Library(lib_nb_books, signup_day, ship_day) 
            lib_info.lbooks = np.array([int (x) for x in next(file).split()]) #How is it working? lbooks is not defined in the class (Laura) You can add attributes to a class dynamically at runtime.
            libraries.append(lib_info)

            #In overall I understand why lib_nb_books, signup_day and ship_day are saved in two different variables
            #Does it have any consequences if we try to optimiwze this part or not at all? (Laura)

        scanned_books = np.ones(nb_books, dtype=bool)
        #Shouldn't we initialize with zeros? (==False) 
        #I use true values because in line 183 use part of this array to filter books that have been scanned already with numpy
        #numpy skips the values whose index corresponds to a 0 (false) in scanned books --> see filter test
        #in the beginning all the books are unscanned. Thus, np.ones()

    start = time.time()
    sorted_libraries = optimizer()
    solutions = allocate_libraries(sorted_libraries)
    print(f'Problem solved in {time.time()-start} seconds')
    create_submission(solutions)

Score: 37.0
Problem solved in 0.0003876686096191406 seconds
Score: 4109300.0
Problem solved in 0.6535475254058838 seconds
Score: 2557627.0
Problem solved in 0.017148733139038086 seconds
Score: 4466967.0
Problem solved in 0.020116567611694336 seconds
Score: 5822900.0
Problem solved in 0.008256912231445312 seconds


IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
""""TEST"""
# Filecrawler
import glob
print(glob.glob("Datasets/*.txt"))

['Datasets/g_score_test.txt', 'Datasets/d_tough_choices.txt', 'Datasets/f_libraries_of_the_world.txt', 'Datasets/e_so_many_books.txt', 'Datasets/b_read_on.txt', 'Datasets/a_example.txt', 'Datasets/c_incunabula.txt']


In [ ]:
"""TEST"""
# Print array
a = [1,2,3,4,5]
' '.join(map(str, a))

'1 2 3 4 5'

In [ ]:
"""TEST"""
# Broadcasted Indexing

lib1 = libraries[0].lbooks
print(scores)
print(lib1)
print(scores[lib1])

[1 2 3 6 5 4]
[0 1 2 3 4]
[1 2 3 6 5]


In [ ]:
"""TEST"""
# Sort while keeping index

a = np.array([[5,2,8,4],[0,1,2,3]])
ind = np.flip(a[0,:].argsort()) # Get indices of first row sorted in descending order
a = a[:,ind]
a

array([[8, 5, 4, 2],
       [2, 0, 3, 1]])

In [ ]:
"""TEST"""
# filter array

a = np.array([[5,2,8,4],[0,1,2,3]])
print(a)
b = np.array([0, 0, 1, 1, 1, 0, 0, 1, 1], dtype=bool)
print(b)
c = b[a[1,:]]
print(c)
d = a[:,c]
print(d)
b[d[1,:]] = 0
print(b)

[[5 2 8 4]
 [0 1 2 3]]
[False False  True  True  True False False  True  True]
[False False  True  True]
[[8 4]
 [2 3]]
[False False False False  True False False  True  True]


In [ ]:
"""TEST"""
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # Division avec arronid
print(x.max())

a = np.arange(3)
# La ligne suivante crée une matrice de taille 3x1
# avec trois lignes et une colonne.
b = np.arange(3)[:, np.newaxis]
a+b
print(a)
print(b)
print(a+b)
print('flip')
z=np.zeros([2,2])
z[1,0]=2
z[0,0]=5
z[0,1]=3
z[1,1]=4
print(z)
print(z.argsort())
print(np.flip(z[0,:].argsort()))
print(z[(z[1,:].argsort())])
print(np.flip(z))

x     = [0 1 2 3]
x + 5 = [5 6 7 8]
x - 5 = [-5 -4 -3 -2]
x * 2 = [0 2 4 6]
x / 2 = [0.  0.5 1.  1.5]
x // 2 = [0 0 1 1]
3
[0 1 2]
[[0]
 [1]
 [2]]
[[0 1 2]
 [1 2 3]
 [2 3 4]]
flip
[[5. 3.]
 [2. 4.]]
[[1 0]
 [0 1]]
[0 1]
[[5. 3.]
 [2. 4.]]
[[4. 2.]
 [3. 5.]]


In [ ]:
"""TEST LAURA"""
import numpy as np
import time
import glob
import os


class Library:
    
    def __init__(self, nb_books, signup_day, ship_day):
        self.nb_books = nb_books
        self.signup_day = signup_day
        self.ship_day = ship_day

def optimizer2(total_days):
    scanned_books = np.zeros(nb_books, dtype=bool)
    overallScore=0
    """Optimization strategy"""
    # Calculate depending on our many days is left
    # For each library, calculate the best score depending on which books are left to scan
    order_to_scan= []

    #Create a sorted table with the score and books in descending orde
    score = np.zeros([2,nb_books])
    score[0,:] = scores
    score[1,:] = np.arange(0,nb_books)

    while(total_days>0):
        #Number of books we can scan per library depending on how many days are left
        nr_books_to_scan = ship_day_list * (total_days - signup_day_list) 
        nr_books_to_scan = nr_books_to_scan.astype(np.int64)
        #Get the sum of scores of each library
        sum_score = np.zeros([2,total_libraries])
        for lib in range(total_libraries):
            books_not_scan_yet = np.where(scanned_books[libraries[lib].lbooks]==False)
            books_not_scan_yet = books_not_scan_yet[0]

            temp = np.zeros([2,len(books_not_scan_yet)])
            temp[0,:] = score[0,books_not_scan_yet]
            temp[1,:] = books_not_scan_yet

            temp_idx = np.flip(temp[0,:].argsort()) # Return sorted indices of book scores in library
            temp_sorted = temp[:, temp_idx]

            sum_score[0,lib] = np.sum(temp_sorted[0,0:nr_books_to_scan[lib]])
            sum_score[1,lib] = lib

        sum_score_idx = np.flip(sum_score[0,:].argsort()) 
        sum_score = sum_score[:, sum_score_idx]

        #Get the best library
        i=0
        best_lib = sum_score[1,i]
        

        while best_lib in order_to_scan:
            i+=1
            best_lib = sum_score[1,i]
            
        best_lib = best_lib.astype(np.int64)
        order_to_scan.append(best_lib)
        scanned_books[libraries[best_lib].lbooks]=1
        overallScore += sum_score[0,best_lib]
        total_days=total_days - libraries[best_lib].signup_day
    
    print(f"overall Score: {overallScore}")

# Parsing
for path in glob.glob("Datasets/*.txt"):
    # Parsing
    file_name = os.path.basename(path)
    with open(path,'r') as file: 
        # get values 
        nb_books, total_libraries, total_days = [int(x) for x in next(file).split()]
        scores = np.array([int (x) for x in next(file).split()])

        #get the information for each library:
        #nb_books: number total of books
        #signup_day: number of day needed to sign up
        #ship_day: number of books per day 
        #lbooks: list of the books in the library

        # Preallocate arrays that will be used for optimization later
        nb_books_list = np.zeros([total_libraries]) 
        signup_day_list = np.zeros([total_libraries])
        ship_day_list = np.zeros([total_libraries])
        libraries = [] # list of all Library objects
        for i in range(total_libraries):
            lib_nb_books, signup_day, ship_day = [int(x) for x in next(file).split()]
            
            # add library specific information to arrays for optimization
            nb_books_list[i] = lib_nb_books
            signup_day_list[i] = signup_day
            ship_day_list[i] = ship_day
            
            # add library specific information to Library object
            lib_info = Library(lib_nb_books, signup_day, ship_day) 
            lib_info.lbooks = np.array([int (x) for x in next(file).split()]) 
            libraries.append(lib_info)
    
    start = time.time()
    print('File:',file_name)
    optimizer2(total_days)
    end = time.time()
    duration = end-start
    print('Problem solved in %f seconds' %duration)









File: g_score_test.txt
overall Score: 27.0
Problem solved in 0.000876 seconds
File: d_tough_choices.txt


KeyboardInterrupt: 